# ドロップアウト (Dropout)

In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.2の大きさのノイズが乗ったデータを考えます。データ数は30にします。
早期終了の例と同様に、scikit learningの関数を使ってデータをトレーニング用と検証用に分割します。

In [ ]:
np.random.seed(0)

nSample = 30

x = np.linspace(-np.pi, np.pi, nSample)
t0 = np.sin(x)
t = t0 + 0.2 * np.random.randn(nSample)

from sklearn.model_selection import train_test_split
x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.2)

# For plot
x_grid = np.linspace(-np.pi, np.pi, 100)

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、隠れ層が5層、ノード数が128の隠れ層を4層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential([
    Dense(128, activation='relu', input_dim=1),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dense(1, activation='linear'),  # ノード数が1の層を追加。活性化関数は線形関数。
])

model.compile(loss='mean_squared_error', optimizer='adam')

#  トレーニング
history = model.fit(
    x=x_train, 
    y=t_train,
    validation_data=(x_valid, t_valid),
    epochs=500,
    verbose=0
)

# ロス関数の推移を取得します
loss_train = history.history['loss']
loss_valid = history.history['val_loss']

# プロット
plt.scatter(x_train, t_train, s=10, c='black', label='data')  # データ点のプロット
plt.plot(x_grid, model.predict(x_grid), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(x)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train, label='loss (train)')
plt.plot(loss_valid, label='loss (valid)')
plt.legend()
plt.show()


## ドロップアウト (Dropout)

ドロップアウトは、学習時に、ランダムにノードを消すという操作をすることで、表現能力を抑制する手法です。
この操作は、アンサンブル学習を擬似的に行っていると見ることもできます。

この操作もKerasの関数を用いることで簡単に実装する事ことができます。

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout

```python
Dropout(rate=0.3)
```
ここで、消すノードの割合を`rate`で指定します。0だと、ノードは全く消されません。1.0だと、全てのノードが消されます。

個々では、MLPの各層の後にドロップアウト層を入れることにします。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential([
    Dense(128, activation='relu', input_dim=1),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dropout(rate=0.3),  # 30%の確率でノードの値を0にする。
    Dense(128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dropout(rate=0.3),  # 30%の確率でノードの値を0にする。
    Dense(128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dropout(rate=0.3),  # 30%の確率でノードの値を0にする。
    Dense(128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dropout(rate=0.3),  # 30%の確率でノードの値を0にする。
    Dense(1, activation='linear'),  # ノード数が1の層を追加。活性化関数は線形関数。
])

model.compile(loss='mean_squared_error', optimizer='adam')

#  トレーニング
history = model.fit(
    x=x_train, 
    y=t_train,
    validation_data=(x_valid, t_valid),
    epochs=500,
    verbose=0
)

# L2を除いたロス関数の推移を取得します
loss_train = history.history['loss']
loss_valid = history.history['val_loss']

# プロット
plt.scatter(x_train, t_train, s=10, c='black', label='data')  # データ点のプロット
x_grid = np.linspace(-np.pi, np.pi, 100)
plt.plot(x_grid, model.predict(x_grid), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(x)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train, label='loss (train)')
plt.plot(loss_valid, label='loss (valid)')
plt.legend()
plt.show()

トレーニングデータに対する誤差関数が大きくふらついていることがわかります。
これは、消されるノードが毎回ランダムに選ばれるためです。
一方で、検証用データではばらつきは比較的小さくなっています。
これは、Kerasの実装では、検証用データを使ってモデルの予測をする際は、全てのノードを使うようになっているためです。

検証用データをドロップアウトの`rate`の値を変えたり、ドロップアウトの位置を変えると、モデルの予測はどのように変化するでしょうか？